# **Tool Calling Agent:**

Tool calling allows a model to detect when one or more tools should be called and respond with the inputs that should be passed to those tools. In an API call, you can describe tools and have the model intelligently choose to output a structured object like JSON containing arguments to call these tools. The goal of tools APIs is to more reliably return valid and useful tool calls than what can be done using a generic text completion or chat API.<br>

We can take advantage of this structured output, combined with the fact that <u>you can **bind multiple tools to a tool calling chat model and allow the model to choose which one to call**, to create an agent that repeatedly calls tools and receives results until a query is resolved.</u><br><br>

<u>Tool Calling is an agent type where an LLM decides **which tools to call** (based on user query), **what inputs to provide**, and how to process results to achieve a task. Tools include APIs, search engines, or calculators.</u>

In [ ]:
# install necessary libaries:

%pip install --upgrade --quiet sentence_transformers
%pip install --upgrade --quiet  langchain langchain-community langchainhub langchain-google-genai langchain-chroma bs4 boto3
%pip install --upgrade --quiet langchain-aws wikipedia youtube-search tavily-python

## **Load LLM & Embeddings::**

In [2]:
# Load the Tokens:

from google.colab import userdata
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY')
os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')

In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.4,
    max_tokens=512,
    timeout=None,
    max_retries=2,
    # other params...
)

print(llm.invoke("hi").content)

Hi there! How can I help you today?



## **Load Embedding from HF:**

In [ ]:
# Get the Embeddings:

from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-MiniLM-L6-v2"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

## **Vector Store:**

In [5]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [7]:
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

In [8]:
# retriever:

retriever = vectorstore.as_retriever(search_kwargs=dict(k=5))

In [ ]:
retriever.get_relevant_documents("what is langchain?")

## **1. Simple Tool Calling Agent:**

### **Get The Tool: One Tool**

* **Tavily-Search-Results**


In [10]:
import os
from google.colab import userdata

TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

In [13]:
# Get the Tavily Search Results Tool:

from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

In [14]:
# use the tool:

tools = [search]

### **Define the Prompt:**

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


prompt

### **Create Agent:**

In [19]:
from langchain.agents import create_tool_calling_agent

In [28]:
# Construct the Tools agent

agent = create_tool_calling_agent(llm, tools, prompt)

In [29]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

### **Run the Agent Executor:**

**AgentExecutor:**<br>
AgentExecutor in LangChain is a component that runs the **agent's logic** **by orchestrating the flow between the LLM, tools, and inputs**. It handles decision-making, tool invocation, and result processing.<br><br>


**Key Features:**<br>

1. **Takes Input:** Accepts the user query and initial parameters.
2. **Execcutes the Agent:** Runs the agent's reasoning logic to select and call tools.
3. **Handles Iteration:** Continues until a final response is generated (or a stopping condition is reached).
4. **Returns Output:** Provides the final processed response.

<br>

The **AgentExecutor** ensures smooth coordination of tasks in a multi-tool environment, making agents efficient and reliable.

In [59]:
from langchain.agents import AgentExecutor

In [30]:
# Run the Agent:

agent_executor.invoke({"input": "what is LangChain?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'what is LangChain?'}`


[{'url': 'https://www.geeksforgeeks.org/introduction-to-langchain/', 'content': 'Data Structures and Algorithms\nML & Data Science\nWeb Development\nLanguages\nInterview Corner\nCS Subjects\nJobs\nPractice\nContests\nIntroduction to Langchain\nLangChain is an open-source framework designed to simplify the creation of applications using large language models (LLMs). LangChain follows a general pipeline where a user asks a question to the language model where the vector representation of the question is used to do a similarity search in the vector database and the relevant information is fetched from the vector database and the response is later fed to the language model. LangChain Key Concepts:\nThe main properties of LangChain Framework are :\nSetting up the environment\nInstallation of langchain is very simple and similar as you install other libraries using the pip

{'input': 'what is LangChain?',
 'output': "LangChain is an open-source framework for developing applications powered by large language models (LLMs). It simplifies the process of creating LLM-backed applications by providing a standard interface, integrations with various data sources and LLM providers, and pre-built components for common LLM tasks.  It allows developers to connect LLMs to other sources of computation or knowledge, such as APIs or web search. Some key features and concepts include:\n\n* **Key Concepts:** Chains, agents, indexes, and callbacks.\n* **Modules:** Prompt templates, LLMs, document loaders, utilities, chains, agents, indexes, memory, and callbacks.\n* **Integrations:**  A wide range of integrations with LLM providers (OpenAI, Hugging Face, etc.), data sources (Google Drive, Wikipedia, etc.), and other tools.\n\nLangChain is used to build a variety of applications, including chatbots, question-answering systems, text summarization tools, and code analysis too

In [31]:
agent_executor.invoke({"input": "Hi. How are you"})



> Entering new AgentExecutor chain...
I'm doing well, thank you for asking. How can I help you today?


> Finished chain.


{'input': 'Hi. How are you',
 'output': "I'm doing well, thank you for asking. How can I help you today?\n"}

## **2. Add Multiple Tools: RAG, YouTube Tools**

### **Get the Tools:**

#### **YouTube Search Tool:**

In [32]:
# Get the YouTube Search Tool:

from langchain_community.tools import YouTubeSearchTool

youtube_tool = YouTubeSearchTool()

#### **Tavily Search Results Tool:**

In [33]:
# Get the Tavily Search Results Tool:

from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults()

#### **RAG Tool:**

In [34]:
# Get the RAG Tool:

from langchain.tools.retriever import create_retriever_tool

In [36]:
retriever_tool = create_retriever_tool(
    retriever=retriever,
    name="langsmith_search",
    description="Search for information about LangSmith. For any questions about LangSmith, you must use this tool!"
)

#### **Get the main tool bycombining all tools:**

In [37]:
tools = [search, retriever_tool, youtube_tool]

### **Create or Define Chat Prompt Template:**

In [38]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant.",
        ),
        ("placeholder", "{chat_history}"),
        ("human", "{input}"),
        ("placeholder", "{agent_scratchpad}"),
    ]
)


# prompt

### **Create Agent:**

In [40]:
from langchain.agents import create_tool_calling_agent, AgentExecutor

In [41]:
agent = create_tool_calling_agent(llm, tools, prompt)

### **Call Agent: For Generating Response**

In [42]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [43]:
agent_executor.invoke({"input": "Hi, how are you?"})



> Entering new AgentExecutor chain...
I'm doing well, thank you for asking. How can I help you today?


> Finished chain.


{'input': 'Hi, how are you?',
 'output': "I'm doing well, thank you for asking. How can I help you today?\n"}

In [44]:
agent_executor.invoke({"input": "What is LangSmith?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'What is LangSmith?'}`


Get started with LangSmith | ü¶úÔ∏èüõ†Ô∏è LangSmith






Skip to main contentLearn the essentials of LangSmith in the new Introduction to LangSmith course!  Enroll for free. API ReferenceRESTPythonSearchRegionUSEUGo to AppQuick StartObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceQuick StartOn this pageGet started with LangSmith
LangSmith is a platform for building production-grade LLM applications.
It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.
With LangSmith you can:

LangSmith + LangChain OSSLangSmith integrates seamlessly with LangChain's open source frameworks langchain and langgraph, with no extra instrum

{'input': 'What is LangSmith?',
 'output': 'LangSmith is a platform for building, testing, evaluating, and deploying large language model (LLM) applications. It integrates with LangChain and provides tools to improve LLM application performance.  It offers features for tracing, debugging, and evaluating LLM workflows. You can find more information in their documentation or on their website.\n'}

In [45]:
agent_executor.invoke({"input": "Give some video related to LangSmith"})



> Entering new AgentExecutor chain...

Invoking: `youtube_search` with `{'query': 'LangSmith, 5'}`


['https://www.youtube.com/watch?v=jx7xuHlfsEQ&pp=ygUJTGFuZ1NtaXRo', 'https://www.youtube.com/watch?v=KcGoJPK-0tY&pp=ygUJTGFuZ1NtaXRo', 'https://www.youtube.com/watch?v=Hab2CV_0hpQ&pp=ygUJTGFuZ1NtaXRo', 'https://www.youtube.com/watch?v=ldBsvhjEREc&pp=ygUJTGFuZ1NtaXRo', 'https://www.youtube.com/watch?v=tFXm5ijih98&pp=ygUJTGFuZ1NtaXRo']Here are some videos related to LangSmith:
https://www.youtube.com/watch?v=jx7xuHlfsEQ&pp=ygUJTGFuZ1NtaXRo
https://www.youtube.com/watch?v=KcGoJPK-0tY&pp=ygUJTGFuZ1NtaXRo
https://www.youtube.com/watch?v=Hab2CV_0hpQ&pp=ygUJTGFuZ1NtaXRo
https://www.youtube.com/watch?v=ldBsvhjEREc&pp=ygUJTGFuZ1NtaXRo
https://www.youtube.com/watch?v=tFXm5ijih98&pp=ygUJTGFuZ1NtaXRo


> Finished chain.


{'input': 'Give some video related to LangSmith',
 'output': 'Here are some videos related to LangSmith:\nhttps://www.youtube.com/watch?v=jx7xuHlfsEQ&pp=ygUJTGFuZ1NtaXRo\nhttps://www.youtube.com/watch?v=KcGoJPK-0tY&pp=ygUJTGFuZ1NtaXRo\nhttps://www.youtube.com/watch?v=Hab2CV_0hpQ&pp=ygUJTGFuZ1NtaXRo\nhttps://www.youtube.com/watch?v=ldBsvhjEREc&pp=ygUJTGFuZ1NtaXRo\nhttps://www.youtube.com/watch?v=tFXm5ijih98&pp=ygUJTGFuZ1NtaXRo\n'}

### **Use Memory or Chat History:**

#### **Custom:**

In [46]:
# Here we pass in an empty list of messages for chat_history (because it is the first message in the chat):

agent_executor.invoke({"input": "hi! my name is bob", "chat_history": []})



> Entering new AgentExecutor chain...
Hi Bob, it's nice to meet you! How can I help you today?


> Finished chain.


{'input': 'hi! my name is bob',
 'chat_history': [],
 'output': "Hi Bob, it's nice to meet you! How can I help you today?\n"}

In [47]:
from langchain_core.messages import AIMessage, HumanMessage

# Customly, provide the chat_history
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
            HumanMessage(content="I am working at Accenture, as a Sr. Data Scientist"),
            AIMessage(content="That's awesome, Bob! It sounds like you have a lot of expertise in data science."),
        ],
        "input": "what's my name and where I am working?",
    }
)



> Entering new AgentExecutor chain...
Your name is Bob, and you work at Accenture as a Senior Data Scientist.


> Finished chain.


{'chat_history': [HumanMessage(content='hi! my name is bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Bob! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='I am working at Accenture, as a Sr. Data Scientist', additional_kwargs={}, response_metadata={}),
  AIMessage(content="That's awesome, Bob! It sounds like you have a lot of expertise in data science.", additional_kwargs={}, response_metadata={})],
 'input': "what's my name and where I am working?",
 'output': 'Your name is Bob, and you work at Accenture as a Senior Data Scientist.\n'}

#### **Real-Time Memory:**

In [49]:
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [52]:
# store the session information in message_history:

message_history = ChatMessageHistory()
message_history

InMemoryChatMessageHistory(messages=[])

In [53]:
# Agent Executor with Memory:

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

**Start Invoking using Memory and Agent Executor:**

In [54]:
agent_with_chat_history.invoke(
    {"input": "hi! I'm Dibyendu, Working as a AI/ML Engineer, but still not getting job"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "firstchat"}},
)



> Entering new AgentExecutor chain...
I understand that you're an AI/ML engineer and are currently looking for a job.  It's a competitive field, but don't get discouraged!  I can offer some help with your job search.  To give you the best advice, could you tell me more about your situation? For example:

* **What kind of AI/ML roles are you targeting?** (e.g., Research, Data Science, Machine Learning Engineering, etc.)
* **What's your experience level?** (e.g., Entry-level, Mid-level, Senior)
* **What are your main skills and strengths?** (e.g., specific programming languages, ML algorithms, cloud platforms, etc.)
* **What have you tried so far in your job search?** (e.g., applying online, networking, attending career fairs)

The more information you can provide, the better I can assist you.  I can help with things like:

* **Optimizing your resume and LinkedIn profile.**
* **Practicing for interviews.**
* **Identifying relevant job opportunities.**
* **Finding resources for skill de

{'input': "hi! I'm Dibyendu, Working as a AI/ML Engineer, but still not getting job",
 'chat_history': [],
 'output': "I understand that you're an AI/ML engineer and are currently looking for a job.  It's a competitive field, but don't get discouraged!  I can offer some help with your job search.  To give you the best advice, could you tell me more about your situation? For example:\n\n* **What kind of AI/ML roles are you targeting?** (e.g., Research, Data Science, Machine Learning Engineering, etc.)\n* **What's your experience level?** (e.g., Entry-level, Mid-level, Senior)\n* **What are your main skills and strengths?** (e.g., specific programming languages, ML algorithms, cloud platforms, etc.)\n* **What have you tried so far in your job search?** (e.g., applying online, networking, attending career fairs)\n\nThe more information you can provide, the better I can assist you.  I can help with things like:\n\n* **Optimizing your resume and LinkedIn profile.**\n* **Practicing for inter

In [55]:
agent_with_chat_history.invoke(
    {"input": "Give me 2 video urls related to AI and ML."},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "firstchat"}},
)



> Entering new AgentExecutor chain...

Invoking: `youtube_search` with `{'query': 'AI and ML, 2'}`


['https://www.youtube.com/watch?v=4RixMPF4xis&pp=ygUJQUkgYW5kIE1M', 'https://www.youtube.com/watch?v=qYNweeDHiyU&pp=ygUJQUkgYW5kIE1M']Here are two YouTube video URLs related to AI and ML:

* https://www.youtube.com/watch?v=4RixMPF4xis&pp=ygUJQUkgYW5kIE1M
* https://www.youtube.com/watch?v=qYNweeDHiyU&pp=ygUJQUkgYW5kIE1M


> Finished chain.


{'input': 'Give me 2 video urls related to AI and ML.',
 'chat_history': [HumanMessage(content="hi! I'm Dibyendu, Working as a AI/ML Engineer, but still not getting job", additional_kwargs={}, response_metadata={}),
  AIMessage(content="I understand that you're an AI/ML engineer and are currently looking for a job.  It's a competitive field, but don't get discouraged!  I can offer some help with your job search.  To give you the best advice, could you tell me more about your situation? For example:\n\n* **What kind of AI/ML roles are you targeting?** (e.g., Research, Data Science, Machine Learning Engineering, etc.)\n* **What's your experience level?** (e.g., Entry-level, Mid-level, Senior)\n* **What are your main skills and strengths?** (e.g., specific programming languages, ML algorithms, cloud platforms, etc.)\n* **What have you tried so far in your job search?** (e.g., applying online, networking, attending career fairs)\n\nThe more information you can provide, the better I can ass

In [57]:
agent_with_chat_history.invoke(
    {"input": "What is my name and where I am working?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "firstchat"}},
)



> Entering new AgentExecutor chain...
You told me your name is Dibyendu and that you are an AI/ML Engineer. You mentioned you are currently looking for a job, so you are not currently employed.


> Finished chain.


{'input': 'What is my name and where I am working?',
 'chat_history': [HumanMessage(content="hi! I'm Dibyendu, Working as a AI/ML Engineer, but still not getting job", additional_kwargs={}, response_metadata={}),
  AIMessage(content="I understand that you're an AI/ML engineer and are currently looking for a job.  It's a competitive field, but don't get discouraged!  I can offer some help with your job search.  To give you the best advice, could you tell me more about your situation? For example:\n\n* **What kind of AI/ML roles are you targeting?** (e.g., Research, Data Science, Machine Learning Engineering, etc.)\n* **What's your experience level?** (e.g., Entry-level, Mid-level, Senior)\n* **What are your main skills and strengths?** (e.g., specific programming languages, ML algorithms, cloud platforms, etc.)\n* **What have you tried so far in your job search?** (e.g., applying online, networking, attending career fairs)\n\nThe more information you can provide, the better I can assist

In [58]:
message_history

InMemoryChatMessageHistory(messages=[HumanMessage(content="hi! I'm Dibyendu, Working as a AI/ML Engineer, but still not getting job", additional_kwargs={}, response_metadata={}), AIMessage(content="I understand that you're an AI/ML engineer and are currently looking for a job.  It's a competitive field, but don't get discouraged!  I can offer some help with your job search.  To give you the best advice, could you tell me more about your situation? For example:\n\n* **What kind of AI/ML roles are you targeting?** (e.g., Research, Data Science, Machine Learning Engineering, etc.)\n* **What's your experience level?** (e.g., Entry-level, Mid-level, Senior)\n* **What are your main skills and strengths?** (e.g., specific programming languages, ML algorithms, cloud platforms, etc.)\n* **What have you tried so far in your job search?** (e.g., applying online, networking, attending career fairs)\n\nThe more information you can provide, the better I can assist you.  I can help with things like:\